# CollectionsApi

CollectionsApi lets you create certain collections:
- An **EntityTypeCollection**: a specific set of entity types.  The example in this notebook is the Questionnaire-collection.  It consists of all entity types that collect information on questionnaires.
- An **EntitySetCollection**: an instantiation of an entity type collection.  This is an actual collection of datasets that you can write/read from.

Creating and changing EntityTypeCollections can only be done by administrators, all other functions are open to all users.

## Notebook goals

**At the end of this notebook, we want to have a collection of Entity Sets for an (to be created) EntityTypeCollection of type `ol.questionnaires` in the Demo Organization**
Creating and modifying EntityTypeCollections can only be done by admins, so as a user, you can skip those sections.

_**EntityTypeCollections (admin only)**_
1. (admin) The goal initially is to create an EntityTypeCollection with the following entity types:
  - `ol.study`
  - `general.person`
  - `ol.survey`
  - `ol.answer`
  - `ol.question`
  - `ol.addresses`
  - `ol.respondswith`
  - `ol.partof`
  - `general.participatedin`


2. (admin) We will add and remove another EntityType to and from the EntityTypeCollection.
3. (admin) We will assign metadata to the EntityTypeCollection

_**EntitySetCollections**_
1. Show different ways to find and read EntityTypeCollections
2. Create two EntitySetCollections
3. Change the template of the EntitySetCollection
4. Assign metadata to the EntitySetCollection
5. Find your EntitySetCollections
6. Remove one of the new EntitySetCollections

## Configuration

To connect to the OpenLattice api, we set up the configuration and load the necessary Api's. 
**jwt**: As authentication, we use a jwt-token.  You can find this token in your account information on the [OpenLattice Gallery App](https://openlattice.com/gallery/#/edit_account)

In [1]:
from pprint import pprint
import openlattice

jwt = "***"
baseurl = 'https://api.openlattice.com'
configuration = openlattice.Configuration()
configuration.host = baseurl
configuration.api_key_prefix['Authorization'] = 'Bearer'
configuration.api_key['Authorization'] = jwt

collectionsApi = openlattice.CollectionsApi(
    openlattice.ApiClient(configuration))
edmApi = openlattice.EdmApi(
    openlattice.ApiClient(configuration))
entitySetsApi = openlattice.EntitySetsApi(
    openlattice.ApiClient(configuration))

org_id = "1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8" # demo organization

## Entity Type Collection

First we will create an entity type collection.  If you are not an admin, please skip this part.

### (admin only) Admin Functions

Method | HTTP request | Description
------------- | ------------- | -------------
**add_type_to_entity_type_collection_template** | **PATCH** /datastore/collections/entity/type/{entityTypeCollectionId}/template | Appends type to template of the specified EntityTypeCollection
**create_entity_type_collection** | **POST** /datastore/collections/entity/type | Creates a new EntityTypeCollection
**delete_entity_type_collection** | **DELETE** /datastore/collections/entity/type/{entityTypeCollectionId} | Deletes the specified EntityTypeCollection
**remove_type_from_entity_type_collection_template** | **DELETE** /datastore/collections/entity/type/{entityTypeCollectionId}/template/{typeId} | Removes a key from an EntityTypeCollection template
**update_entity_type_collection_metadata** | **PATCH** /datastore/collections/entity/type/{entityTypeCollectionId} | Updates metadata of the specified EntityTypeCollection

#### Creating and deleting an EntityTypeCollection

First, we want to define a **CollectionTemplateType**, which means we assign metadata to the EntityTypes we want to use.  The following arguments can be given to a CollectionTemplateType.  Most importantly, they contain an EntityTypeId and a name.  All other fields are optional.

The reason we are not just defining EntityTypes is because this way, we can allow multiple of the same EntityTypes to be present in a collection.  In another (future?) context, we might have 2 sets of people: (1) participants and (2) researchers. 

`openlattice.CollectionTemplateType`

Name | Type | Description
------------ | ------------- | -------------
**id** | **str** | An optional UUID for the collection template type.
**name** | **str** | The unique name of the collection template type.
**title** | **str** | The friendly name for the collection template type.
**description** | **str** | A description of the collection template type.
**entity_type_id** | **str** | The id of the entity type that this collection template type wraps.

Below we create the **CollectionTemplateType**'s for our questionnaire EntityTypeCollection.

In [2]:
collectionTemplateTypes = [
    openlattice.CollectionTemplateType(
        entity_type_id = edmApi.get_entity_type_id(
            namespace = "ol", name = "study"),
        name = "studies",
        title = "Studies",
        description = "A collection of all studies run with different questionnaires."      
    ),
    openlattice.CollectionTemplateType(
        entity_type_id = edmApi.get_entity_type_id(
            namespace = "general", name = "person"),
        name = "participants",
        title = "Participants",
        description = "Participants in different studies and questionnaires."      
    ),
    openlattice.CollectionTemplateType(
        entity_type_id = edmApi.get_entity_type_id(
            namespace = "ol", name = "survey"),
        name = "questionnaires",
        title = "Questionnaires",
        description = "Different questionnaires as part of the studies."      
    ),
    openlattice.CollectionTemplateType(
        entity_type_id = edmApi.get_entity_type_id(
            namespace = "ol", name = "question"),
        name = "questions",
        title = "Questions",
        description = "Questions asked in the questionnaires and their metadata.  These fields represent the columns of the output data."      
    ),
    openlattice.CollectionTemplateType(
        entity_type_id = edmApi.get_entity_type_id(
            namespace = "ol", name = "answer"),
        name = "answers",
        title = "Answers",
        description = "Answers provided to the questions asked."      
    ),
    openlattice.CollectionTemplateType(
        entity_type_id = edmApi.get_entity_type_id(
            namespace = "ol", name = "addresses"),
        name = "addresses",
        title = "Addresses"
    ),
    openlattice.CollectionTemplateType(
        entity_type_id = edmApi.get_entity_type_id(
            namespace = "ol", name = "respondswith"),
        name = "responds_with",
        title = "Responds With"
    ),
    openlattice.CollectionTemplateType(
        entity_type_id = edmApi.get_entity_type_id(
            namespace = "ol", name = "partof"),
        name = "part_of",
        title = "Part Of"
    ),
    openlattice.CollectionTemplateType(
        entity_type_id = edmApi.get_entity_type_id(
            namespace = "general", name = "participatedin"),
        name = "participated_in",
        title = "Participated In"
    )    
]

With these defined CollectionTemplateTypes, we can define the **EntityTypeCollection**.


**Note:** we're calling it `test.questionnaires` with the goal of removing it, so that we can re-run this notebook.

Name | Type | Description
------------ | ------------- | -------------
**id** | **str** | An optional UUID for the entity type collection.
**type** | **FullQualifiedName** |  | [optional] 
**title** | **str** | The friendly name for the entity type collection.
**description** | **str** | A description of the entity type collection.
**schemas** | **list[FullQualifiedName**] | A list of schemas the entity type collection should belong to.
**template** | **list[CollectionTemplateType]** | A set of CollectionTemplateType objects, which describe the entity types involved in the entity type collection and the purposes they serve

In [3]:
etc = openlattice.EntityTypeCollection(
    type = openlattice.FullQualifiedName(
        namespace = "test", 
        name = "questionnaires"),
    title = "Questionnaires",
    description = "Collection of EntityTypes for questionnaires",
    schemas = [],
    template = collectionTemplateTypes
)

Finally, we can create the EntityTypeCollection:

In [4]:
etc_id = collectionsApi.create_entity_type_collection(etc)
print(etc_id)

7a3b7962-f5eb-4c3c-8351-2663500ab439


When successful, we get the EntityTypeCollectionId.
We can also delete it again, and recreate:

In [5]:
collectionsApi.delete_entity_type_collection(etc_id)
etc_id = collectionsApi.create_entity_type_collection(etc)
print(etc_id)

36b919d7-cf2b-426d-8ed2-ccfb4d985362


#### Looking at the collection

We can query the collection using `get_entity_type_collection` (more details below for users).  We're just showing the description.  Print out for more details.

In [6]:
etc = collectionsApi.get_entity_type_collection(etc_id)
print(etc.description)

Collection of EntityTypes for questionnaires


#### Adding and removing entity types from the collection

We are now going to add `ol.surveymetadata` and remove it again.

In [7]:
etid = edmApi.get_entity_type_id(namespace = "ol", name = "surveymetadata")

In [8]:
collectionsApi.add_type_to_entity_type_collection_template(
    etc_id,
    openlattice.CollectionTemplateType(
        entity_type_id = etid,
        name = "survey_metadata",
        title = "Survey Metadata"
    )
)

If we now print out all titles of the templates, we can see **Survey Metadata**.

In [9]:
etc = collectionsApi.get_entity_type_collection(etc_id)
print([x.title for x in etc.template])

['Studies', 'Participants', 'Questionnaires', 'Questions', 'Answers', 'Addresses', 'Responds With', 'Part Of', 'Participated In', 'Survey Metadata']


To remove the entityType, we need to find it's unique id.  This is not just the entityTypeId, since that can correspond to multiple instantiations of the same entityType.

In [10]:
collection_template_id = [x.id for x in etc.template if x.name == "survey_metadata"][0]
print(collection_template_id)

8192bb3a-590a-4195-bf80-d6e74bf4938c


In [11]:
collectionsApi.remove_type_from_entity_type_collection_template(etc_id, collection_template_id)

After removing the collectionTemplateType, we can see that **Survey Metadata** is not in the collection anymore.

In [12]:
etc = collectionsApi.get_entity_type_collection(etc_id)
print([x.title for x in etc.template])

['Studies', 'Participants', 'Questionnaires', 'Questions', 'Answers', 'Addresses', 'Responds With', 'Part Of', 'Participated In']


#### Changing metadata on the Collection

To change an EntityTypeCollection, we need to provide the EntityTypeCollectionId, and a general MetadataUpdate-argument.

In [13]:
new_description = "Questionnaires EntityTypeCollection"

collectionsApi.update_entity_type_collection_metadata(
    entity_type_collection_id = etc_id,
    metadata_update = openlattice.MetadataUpdate(
        description = new_description
    )
)

When we call again the EntityTypeCollection, we can indeed see the description has changed.

In [14]:
etc = collectionsApi.get_entity_type_collection(etc_id)
etc.description

'Questionnaires EntityTypeCollection'

### User Functions

As a user, to browse around EntityTypeCollections, there are two functions.  

Method | HTTP request | Description
------------- | ------------- | -------------
[**get_entity_type_collection**](CollectionsApi.md#get_entity_type_collection) | **GET** /datastore/collections/entity/type/{entityTypeCollectionId} | Returns the EntityTypeCollection object for a given id
[**get_all_entity_type_collections**](CollectionsApi.md#get_all_entity_type_collections) | **GET** /datastore/collections/entity/type | Returns all EntityTypeCollection objects

Getting all entitytypes is a big object, so we only print out parts from it.

In [15]:
etcs = collectionsApi.get_all_entity_type_collections()

In [16]:
pprint({x.title: {"id": x.id, "entity_types": [y.name for y in x.template]} for x in etcs})

{'Jail related entity sets': {'entity_types': ['person_collection',
                                               'jailstay_collection',
                                               'jailstaylength_collection',
                                               'facility_collection',
                                               'case_collection',
                                               'persondetails_collection',
                                               'persondetailscriminaljustice_collection',
                                               'education_collection',
                                               'subjectof_collection',
                                               'locatedat_collection',
                                               'appearsin_collection',
                                               'has_collection',
                                               'oflength_collection'],
                              'id': '1c3a5e56-c9f2-4bf7-84a8-9fd9

To query a specific EntityTypeCollection, you will need the id.

In [17]:
etc_id = [x.id for x in etcs if x.title == "Questionnaires"][0]
etc = collectionsApi.get_entity_type_collection(etc_id)
pprint(etc)

{'description': 'Questionnaires EntityTypeCollection',
 'id': '36b919d7-cf2b-426d-8ed2-ccfb4d985362',
 'schemas': [],
 'template': [{'description': 'A collection of all studies run with different '
                              'questionnaires.',
               'entity_type_id': '80c86a96-0e3f-46eb-9fbb-60d9174566a5',
               'id': '57f21f1b-dbab-43e1-a199-ef7dd12c9c34',
               'name': 'studies',
               'title': 'Studies'},
              {'description': 'Participants in different studies and '
                              'questionnaires.',
               'entity_type_id': '31cf5595-3fe9-4d3e-a9cf-39355a4b8cab',
               'id': '252c2d61-9387-4153-9e12-93eb08ad28c7',
               'name': 'participants',
               'title': 'Participants'},
              {'description': 'Different questionnaires as part of the '
                              'studies.',
               'entity_type_id': 'a5a5d8a0-4433-422c-b096-719b8c573f29',
               'id': '355db

## Entity Set Collection

Now that we went over creating and changing the EntityTypeCollection, which basically serves as a blueprint for a combination of EntityTypes, we can instantiate the entity set collection.

Method | HTTP request | Description
------------- | ------------- | -------------
**create_entity_set_collection** | **POST** /datastore/collections/entity/set | Creates a new EntitySetCollection
**delete_entity_set_collection** | **DELETE** /datastore/collections/entity/set/{entitySetCollectionId} | Deletes the specified EntitySetCollection
**get_all_entity_set_collections** | **GET** /datastore/collections/entity/set | Returns all EntitySetCollection objects
**get_entity_set_collection** | **GET** /datastore/collections/entity/set/{entitySetCollectionId} | Returns the EntitySetCollection object for a given id
**get_entity_set_collections_of_type** | **GET** /datastore/collections/entity/type/entity/set/{entitySetCollectionId} | Returns all authorized EntitySetCollections for a given EntityTypeCollection id
**update_entity_set_collection_metadata** | **PATCH** /datastore/collections/entity/set/{entitySetCollectionId} | Updates metadata of the specified EntitySetCollection
**update_entity_set_collection_template** | **PATCH** /datastore/collections/entity/set/{entitySetCollectionId}/template | Updates template of the specified EntitySetCollection

### Create an EntitySetCollection

First, we create the EntitySetCollection metadata, and with that object create the EntitySetCollection.

The `template`-argument is a mapping from `{collectionTemplateType: entitySetId}`.  If you already have existing entitySets that you want to add to the new EntitySetCollection, you can fill those in in the template argument.
If you set this argument to an empty dictionary, all entitysets will be created.

In [18]:
esc = openlattice.EntitySetCollection(
    title = "My Test Questionnaires",
    name = "my_test_questionnaires",
    description = "A temporary EntitySetCollection around the test.questionnaire entityTypeCollection",
    entity_type_collection_id = etc_id,
    contacts = ["demo@openlattice.com"],
    organization_id = org_id, # demo organization
    template = {}
)

In [19]:
# this can take a while
esc_id = collectionsApi.create_entity_set_collection(esc)

### Get an EntitySetCollection

When querying the entitySetCollection, you can see that the `id` and `template` has been filled out.
The template has a mapping of the templateCollection to the newly created EntitySets.

In [20]:
esc = collectionsApi.get_entity_set_collection(esc_id)

In [21]:
esc

{'contacts': ['demo@openlattice.com'],
 'description': 'A temporary EntitySetCollection around the test.questionnaire '
                'entityTypeCollection',
 'entity_type_collection_id': '36b919d7-cf2b-426d-8ed2-ccfb4d985362',
 'id': '7ecab07a-a356-48db-8b76-47f727d30d0c',
 'name': 'my_test_questionnaires',
 'organization_id': '1d5aa1f4-4d22-46a5-97cd-dcc6820e7ff8',
 'template': {'1d3b4158-e0cf-4f0a-b83e-b46265d74a4f': '5a901798-bb93-4e43-8fea-9008cbd5aa81',
              '252c2d61-9387-4153-9e12-93eb08ad28c7': '88acff45-524c-4533-9cdf-3bc25601411b',
              '355db194-3474-41e6-b2f7-1083baa254a8': 'a4bdd3a1-1af6-4d24-ac1b-f60a3a51a882',
              '57f21f1b-dbab-43e1-a199-ef7dd12c9c34': '5e88be28-6061-4ca1-b8f8-0ce826d96ea9',
              '741d563e-8872-476d-94e5-bdb3d557bf83': '3baaf5e8-c06c-47ac-a032-690769ce20f6',
              '8f615cb6-1aa0-4355-9c4b-0f2a63df3c07': '523d7413-b08c-463f-8498-bc79ef052199',
              '9786d32d-351f-4798-8e81-cc7cda3067ea': 'c0a44cd2-

Below we create a dictionary `transfo` that has a mapping from the CollectionTemplate to the entityTypeTitle.  This we can then use to find all parts of the EntitySetCollection.

In [22]:
transfo = {x.id: edmApi.get_entity_type(x.entity_type_id).title for x in etc.template}

In [23]:
{transfo[tmpl_id]: entitySetsApi.get_entity_set(es_id).name \
     for tmpl_id, es_id in esc.template.items()}

{'Addresses (assn)': 'my_test_questionnaires_addresses',
 'Question': 'my_test_questionnaires_questions',
 'Participated in': 'my_test_questionnaires_participated_in',
 'Questionnaire / Survey': 'my_test_questionnaires_questionnaires',
 'Person': 'my_test_questionnaires_participants',
 'Responds With': 'my_test_questionnaires_responds_with',
 'Answer': 'my_test_questionnaires_answers',
 'Part of': 'my_test_questionnaires_part_of',
 'Study Data Collection Event': 'my_test_questionnaires_studies'}

### Get all entity set collections

In [24]:
escs = collectionsApi.get_all_entity_set_collections()
{x.id: x.title for x in escs}

{'7ecab07a-a356-48db-8b76-47f727d30d0c': 'My Test Questionnaires',
 'adc75807-3f06-4cf3-b4b3-d91be7aab440': 'Demo data for Jail'}

### Get all entity sets of a certain type

If we only want to see entity set collections of a certain type of entityTypeCollections, we can query only the specific entityTypeCollection.

In [25]:
escs_type = collectionsApi.get_entity_set_collections_of_type(etc_id)
{x.id: x.title for x in escs_type}

{'7ecab07a-a356-48db-8b76-47f727d30d0c': 'My Test Questionnaires'}

### Updating EntitySetCollectionMetadata

We can update the metadata of our EntitySetCollection like the with EntityTypeCollection.

In [26]:
new_description = "Demo Questionnaires EntitySetCollection"

collectionsApi.update_entity_set_collection_metadata(
    entity_set_collection_id = esc_id,
    metadata_update = openlattice.MetadataUpdate(
        description = new_description
    )
)

In [27]:
esc = collectionsApi.get_entity_set_collection(esc_id)
esc.description

'Demo Questionnaires EntitySetCollection'

### Changing EntitySets in the collection

If you want to change a certain entity set in the collection, you can swap out entitysets.  For example, if we want to change the person-dataset with a new one, we can use this function.

This will also be very relevant when new EntityTypes are added to the EntityTypeCollection.

Below, we make a new entityset of type `general.person` and change and revert adding this entityset to the collection.

In [28]:
es = openlattice.EntitySet(
    organization_id = org_id,
    entity_type_id = edmApi.get_entity_type_id(
        namespace = "general", 
        name = "person"),
    name = "temporary_entity_set",
    title = "A Temporary Entity Set for Demo Purposes",
    contacts = ['demo@openlattice.com']
)

entitySetsApi.create_entity_sets([es])
es_id = entitySetsApi.get_entity_set_id(es.name)
print(es_id)

deb1adf6-9f03-49bc-b2b1-0b0d53662f5d


To know which part of the collection we want to change, we need to dig into the entityTypeCollection and find the id for the `general.person`-part.  If you have multiple of the same entityType in the same collection, you probably want to do some manual intervention.

In [29]:
etc = collectionsApi.get_entity_type_collection(etc_id)

In [30]:
person_id = [x.id for x in etc.template if x.name == 'participants'][0]
print(person_id)

252c2d61-9387-4153-9e12-93eb08ad28c7


In [31]:
collectionsApi.update_entity_set_collection_template(
    esc_id, 
    request_body = {person_id: es_id}
)

Now if we look at the entity set collection, we can see that this entityset is now in the collection.

In [32]:
esc = collectionsApi.get_entity_set_collection(esc_id)

In [33]:
[entitySetsApi.get_entity_set(v).name for k,v in esc.template.items()]

['my_test_questionnaires_addresses',
 'my_test_questionnaires_questions',
 'my_test_questionnaires_participated_in',
 'my_test_questionnaires_questionnaires',
 'temporary_entity_set',
 'my_test_questionnaires_responds_with',
 'my_test_questionnaires_answers',
 'my_test_questionnaires_part_of',
 'my_test_questionnaires_studies']

## Clean up

In the last part, we will clean up whatever new stuff we made.

**5.1. Undo swapping out entity set and remove temp entity set**

In [34]:
old_esid = entitySetsApi.get_entity_set_id("my_test_questionnaires_participants")

collectionsApi.update_entity_set_collection_template(
    esc_id, 
    request_body = {person_id: old_esid}
)

entitySetsApi.delete_entity_set(es_id)

**5.2. Get latest entity set collection and remove all entity sets**

In [35]:
esc = collectionsApi.get_entity_set_collection(esc_id)

In [36]:
for es_id in esc.template.values():
    entitySetsApi.delete_entity_set(es_id)

**5.3. Remove entity set collection**

In [37]:
collectionsApi.delete_entity_set_collection(esc_id)

**5.4. Remove entity type collection**

In [38]:
collectionsApi.delete_entity_type_collection(etc_id)